<a href="https://colab.research.google.com/github/iLullZz/Colaboratory/blob/main/hamonepuGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
pip install Pillow

In [19]:
pip install Numpy

In [20]:
pip install requests requests_oauthlib

In [21]:
pip install tweepy

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import os, datetime, csv, json
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageEnhance

class CreateImage :
	def __init__(self, size, log) :
		self.imgSize = size;
		self.img = Image.new("RGB", size, (0, 0, 0))
		self.log = log
		self.text = { "font" : None, "size" : None, "rgba" : None }
    
	def addImage(self, pos = (0, 0) , size = None) :
		size = size if size else self.imgSize
		self.img.paste(self.nextImg, pos, self.nextImg)
		
	def addNext(self, r, g, b) :
		self.nextImg = Image.new("RGBA", self.imgSize, (r, g, b, 0))
		return self

	def nextImage(self, src, pos = (0, 0) , size = None, alpha = None) :
		size = size if size else self.imgSize
		img = Image.open(src).resize(size)
		alphaImg = Image.open(alpha).resize(size).convert("RGBA") if alpha else img
		self.nextImg.paste(img, pos, alphaImg)
		del img, alphaImg
		return self

	def nextText(self, text, pos, rgba = None, font = None, size = None) : 
		rgbaSet = rgba if rgba else self.text["rgba"]
		color = (rgbaSet[0], rgbaSet[1], rgbaSet[2], round(rgbaSet[3] * 255))
		fontSet = ImageFont.truetype(font = (font if font else self.text["font"]), size = (size if size else self.text["size"]), encoding = "unic")
		
		draw = ImageDraw.Draw(self.nextImg)
		draw.text(pos, text = text, font = fontSet, fill = color)
		del draw
		return self


	def nextImage(self, src, pos = (0, 0) , size = None, alpha = None) :
		size = size if size else self.imgSize
		img = Image.open(src).resize(size)
		alphaImg = Image.open(alpha).resize(size).convert("RGBA") if alpha else img
		self.nextImg.paste(img, pos, alphaImg)
		del img, alphaImg
		return self

	def nextGamma(self, gamma) :
		lookup = [min(round((i / 255) ** (1 / gamma) * 255), 255) for i in range(256)]
		imgArray = np.array(self.nextImg)
		toGamma = np.vectorize(lambda x : lookup [x])
		imgArray = toGamma(imgArray)
		self.nextImg = Image.fromarray(np.uint8(imgArray))
		del imgArray
		return self

	def getImageSize(self, imgSrc, size, angle) :
		img = Image.open(imgSrc)
		if angle == "width" :
			return (size , int(round(img.height / img.width * size)))
		else :
			return (int(round(img.width / img.height * size)), size)


	def save(self, filePath, fileType) :
		self.img.save(filePath, fileType)

In [32]:
import requests
from requests_oauthlib import OAuth1
import json

import sys
ROOTPATH ='drive/My Drive/アカペラ/Work/ハモネプ/'
sys.path.append(ROOTPATH)

import my_module

from my_module import config

CK = config.CONSUMER_KEY
CS = config.CONSUMER_SECRET
AT = config.ACCESS_TOKEN
ATS = config.ACCESS_SECRET

class Twitter :
  URL = {
    "update" : "https://api.twitter.com/1.1/statuses/update.json",
    "media" : "https://upload.twitter.com/1.1/media/upload.json"
        }

  def __init__(self) :
    self.OAuth = OAuth1(CK, CS, AT, ATS)
    self.log = log

  def mediaUpload(self, src) :
    # media = open('/content/drive/My Drive/アカペラ/Work/ハモネプ/new_img/" + time["datetime"].strftime("%Y-%m-%d.%H-%M-%S") + ".png"' "rb")
    media = open(src, "rb")

    params = { "media" : media }
    request = requests.post(Twitter.URL["media"], files = params, auth = self.OAuth)
    requestJson = json.loads(request.text)
    if request.status_code == 200 :
      self.log.add("画像のアップロードに成功しました")
      self.log.add("・media-id：%s" % requestJson["media_id"])
      return {"state" : True, "media-id" : requestJson["media_id"] }
    else :
      errorMsg = requestJson["errors"][0]
      self.log.add("画像のアップロードに失敗しました・HTTPステータスコード：%s" % request.status_code)
      self.log.add("・エラーコード：%s" % errorMsg["code"])
      self.log.add("・エラーメッセージ：%s" % errorMsg["message"])
      return { "state" : True, "status-code" : request.status_code, "code" : errorMsg["code"], "message" : errorMsg["message"]}

  def update(self, txt, mediaId) :
    params = { "status" : txt, "media_ids" : (mediaId) }
    request = requests.post(Twitter.URL["update"], params = params, auth = self.OAuth)
    requestJson = json.loads(request.text)

    if request.status_code == 200 :
      self.log.add("投稿に成功しました")
      self.log.add("・id：%s" % requestJson["id"])
      self.log.add("・投稿内容：%s" % requestJson["text"])
      return True;
    else  :
      errorMsg = requestJson["errors"][0]
      self.log.add("投稿に失敗しました")
      self.log.add("・HTTPステータスコード\t：%s" % (request.status_code))
      self.log.add("・エラーコード：%s" % errorMsg["code"])
      self.log.add("・エラーメッセージ：%s" % errorMsg["message"])
      return { "state" : True, "status-code" : request.status_code, "code" : errorMsg["code"], "message" : errorMsg["message"] }
      

In [25]:
class Log :
	def __init__(self) :
		self.text = ""
	def add(self, text) :
		self.text += text + "\r\n"
		print(text)
	def save(self, path) :
		with open(path, "a", newline="") as file :
			file.write(self.text + "\r\n")

In [26]:
	#ログ生成
	log = Log()

In [35]:
import re
import urllib
import tweepy

URL = {
  "update" : "https://api.twitter.com/1.1/statuses/update.json",
  "media" : "https://upload.twitter.com/1.1/media/upload.json"
      }

auth = tweepy.OAuthHandler(CK,CS)

class Listener(tweepy.StreamListener):
  #自動でフォロー
  def on_event(self, event):
    if event.event == 'follow':
      if event.source["id"] != api.me().id:
        api.create_friendship(event.source["id"])

In [33]:
if __name__ == "__main__":

	log = Log()
	# log.add("実行日時：" + time["date"]  + time["time"])
 
	listener = Listener()
	stream = tweepy.Stream(auth, listener)
	stream.userstream()

 
	#画像生成
	imgSize = (1122, 1114)
	White4textcolor = (255, 255, 255, 1.0)
	Black4textcolor = (0, 0, 0, 1.0)

	#フォント取得
	fontSrc = {
		"Gosicc" :"/content/drive/My Drive/アカペラ/Work/ハモネプ/HGRGE.TTC",
	}

	#画像取得
	imgSrc = {
		"template" : "/content/drive/My Drive/アカペラ/Work/ハモネプ/ハモネプテンプレ.png",
		"partyparott" : "/content/drive/My Drive/背景素材/partyparott.png",
	}
	#時間取得
	time = { "datetime" : datetime.datetime.now() }
	time["year"] = time["datetime"].year
	time["month"] = time["datetime"].month
	time["day"] = time["datetime"].day
	time["time"] = time["datetime"].strftime("%H:%M:%S:%f")
	time["date"] = "%d.%d/%d" % (time["year"], time["month"], time["day"])


	#リサイズ
	im = Image.open('/content/drive/My Drive/背景素材/partyparott.png').convert("RGBA")
	im.thumbnail((1122,756),Image.ANTIALIAS)
  # img_resize = img.resize((1122, 756))
	mediaPath_R= "/content/drive/My Drive/アカペラ/Work/ハモネプ/new_img/"+"Resize"+".png"
	im.save(mediaPath_R, "PNG")
	imgSrc["Resize"] = "/content/drive/My Drive/アカペラ/Work/ハモネプ/new_img/Resize.png"


	img = CreateImage(imgSize,log)
	img.addNext(255, 255, 255).nextImage(imgSrc["Resize"],(0,100),(1122, 756)).nextGamma(0.8).addImage((0,108))
	img.addNext(255, 255, 255).nextImage(imgSrc["template"]).addImage()

	#文字入れ系
	FtSz = 80
	BandName = "PartyParrot"
	font = ImageFont.truetype(fontSrc["Gosicc"], size=FtSz)

	img4b = Image.open('/content/drive/My Drive/アカペラ/Work/ハモネプ/ハモネプテンプレ.png')

	draw = ImageDraw.Draw(img4b)
	draw_text_width, draw_text_height= draw.textsize(BandName, font=font)
	start_X_point = img4b.size[0]/ 2 - draw_text_width / 2
 
#エントリーNo
	img.text["rgba"] = White4textcolor
	img.text["font"] = fontSrc["Gosicc"]
	img.addNext(255, 255, 255).nextText("エントリーNo.XX", (250,51), size = 80).addImage()
#バンド名
	img.text["rgba"] = Black4textcolor
	img.text["font"] = fontSrc["Gosicc"]
	img.addNext(255, 255, 255).nextText(BandName, (start_X_point,980), size = FtSz).addImage()

#出力
mediaPath = "/content/drive/My Drive/アカペラ/Work/ハモネプ/new_img/" + time["datetime"].strftime("%Y-%m-%d.%H-%M-%S") + ".png"
img.save(mediaPath, "PNG")

twitter = Twitter()
media = twitter.mediaUpload(mediaPath)
if media["state"] :
	twitter.update("PythonからTwitter APIを動かすあれ", media["media-id"])
 
log.save("/content/drive/My Drive/アカペラ/Work/ハモネプ/new_img//data.log")

画像のアップロードに成功しました
・media-id：1380391569866555394
投稿に成功しました
・id：1380391572093726723
・投稿内容：PythonからTwitter APIを動かすあれ https://t.co/8oBHWKwblt
